In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./data\gender_submission.csv
./data\test.csv
./data\train.csv


In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [3]:
train_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train_df.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [5]:
# from copy import deepcopy
# from numpy import isnan
# def transform_nominal(df, columns=None, include_nan=False):
#     df = deepcopy(df)
#     if not columns:
#         columns = list(df.dtypes[df.dtypes=='object'].index)
    
#     for column in columns:
#         column_series = df[column]
#         for nominal_value in column_series.unique():
#             if isinstance(nominal_value, float) and isnan(nominal_value):
#                 if include_nan:
#                     nominal_value = 'nan'
#                 else:
#                     continue
# #             print(nominal_value)
# #             print(isinstance(nominal_value, float))
# #             print(isnan(nominal_value))
# #             print(type(nominal_value), isnan(nominal_value))
#             df[column + '_' + nominal_value] = column_series.apply(lambda value: 1 if value == nominal_value else 0)
#     return df
            
# transform_nominal(train_df, columns=['Embarked']).head()

In [6]:
def one_hot(df, cols):
    """
    @param df pandas DataFrame
    @param cols a list of columns to encode 
    @return a DataFrame with one-hot encoding
    """
    for each in cols:
        dummies = pd.get_dummies(df[each], prefix=each, drop_first=False)
        df = pd.concat([df, dummies], axis=1)
    df = df.drop(cols, axis=1)
    return df

In [7]:
def add_rank(df):
    df['Rank'] = df['Name'].str.extract(r'\b(\w+)\.')
    df.drop(['Name'], axis=1, inplace=True)
    return df

In [8]:
train_df.fillna(np.nan)
train_df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [9]:
test_df[np.isnan(test_df.Fare)]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
152,1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


In [10]:
test_df.fillna(np.nan)
test_df.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [11]:
add_rank(train_df).head()
add_rank(test_df).head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Rank
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q,Mr
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S,Mrs
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q,Mr
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S,Mr
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S,Mrs


In [12]:
test_df.loc[np.isnan(test_df.Fare), 'Fare'] = test_df[(test_df.Embarked=='S') & (test_df.Rank == 'Mr')]['Fare'].mean()

In [13]:
dataset =  pd.concat(objs=[train_df, test_df], axis=0, sort=True)

In [14]:
dataset.columns

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Parch', 'PassengerId', 'Pclass',
       'Rank', 'Sex', 'SibSp', 'Survived', 'Ticket'],
      dtype='object')

In [15]:
rank_age_map = dataset.groupby('Rank')['Age'].agg('mean').to_dict()
rank_age_map

{'Capt': 70.0,
 'Col': 54.0,
 'Countess': 33.0,
 'Don': 40.0,
 'Dona': 39.0,
 'Dr': 43.57142857142857,
 'Jonkheer': 38.0,
 'Lady': 48.0,
 'Major': 48.5,
 'Master': 5.482641509433963,
 'Miss': 21.774238095238097,
 'Mlle': 24.0,
 'Mme': 24.0,
 'Mr': 32.25215146299484,
 'Mrs': 36.99411764705882,
 'Ms': 28.0,
 'Rev': 41.25,
 'Sir': 49.0}

In [16]:
dataset.loc[dataset.Age.isnull(), :].head()

,Age,Cabin,Embarked,Fare,Parch,PassengerId,Pclass,Rank,Sex,SibSp,Survived,Ticket
5,NaN,NaN,Q,8.4583,0,6,3,Mr,male,0,0.0,330877
17,NaN,NaN,S,13.0000,0,18,2,Mr,male,0,1.0,244373
19,NaN,NaN,C,7.2250,0,20,3,Mrs,female,0,1.0,2649
26,NaN,NaN,C,7.2250,0,27,3,Mr,male,0,0.0,2631
28,NaN,NaN,Q,7.8792,0,29,3,Miss,female,0,1.0,330959


In [17]:
for ind, row in dataset.loc[dataset.Age.isnull(), :].iterrows():
    dataset.loc[ind, 'Age'] = rank_age_map[row['Rank']]

In [18]:
dataset.isnull().sum()

Age               0
Cabin          1014
Embarked          2
Fare              0
Parch             0
PassengerId       0
Pclass            0
Rank              0
Sex               0
SibSp             0
Survived        418
Ticket            0
dtype: int64

In [19]:
from sklearn.preprocessing import normalize
def extract_features(df):
    df = df[['PassengerId', 'Survived', 'Pclass', 'Rank', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']]
    
    df.loc[:, 'Sex'] = df['Sex'].map({'male': 1, 'female': 0})
    
    df = one_hot(df, ['Embarked', 'Rank'])
    
    df_features = df.drop(['PassengerId', 'Survived'], axis=1)
    df_features=(df_features-df_features.mean())/df_features.std()
    df = pd.concat([df_features, df[['PassengerId', 'Survived']]], axis=1)
    
    return df
dataset = extract_features(dataset)

C:\Users\henry\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [20]:
dataset.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Rank_Capt,...,Rank_Miss,Rank_Mlle,Rank_Mme,Rank_Mr,Rank_Mrs,Rank_Ms,Rank_Rev,Rank_Sir,PassengerId,Survived
0,0.841595,0.743213,-0.633572,0.481104,-0.444829,-0.503222,-0.509575,-0.321917,0.657142,-0.027639,...,-0.497660,-0.039103,-0.027639,0.853602,-0.420741,-0.039103,-0.078386,-0.027639,1,0.0
1,-1.545507,-1.344482,0.646860,0.481104,-0.444829,0.734378,1.960920,-0.321917,-1.520578,-0.027639,...,-0.497660,-0.039103,-0.027639,-1.170611,2.374944,-0.039103,-0.078386,-0.027639,2,1.0
2,0.841595,-1.344482,-0.313464,-0.478904,-0.444829,-0.490176,-0.509575,-0.321917,0.657142,-0.027639,...,2.007868,-0.039103,-0.027639,-1.170611,-0.420741,-0.039103,-0.078386,-0.027639,3,1.0
3,-1.545507,-1.344482,0.406779,0.481104,-0.444829,0.382941,-0.509575,-0.321917,0.657142,-0.027639,...,-0.497660,-0.039103,-0.027639,-1.170611,2.374944,-0.039103,-0.078386,-0.027639,4,1.0
4,0.841595,0.743213,0.406779,-0.478904,-0.444829,-0.487761,-0.509575,-0.321917,0.657142,-0.027639,...,-0.497660,-0.039103,-0.027639,0.853602,-0.420741,-0.039103,-0.078386,-0.027639,5,0.0


In [21]:
test_df.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Rank
0,892,3,male,34.5,0,0,330911,7.8292,NaN,Q,Mr
1,893,3,female,47.0,1,0,363272,7.0000,NaN,S,Mrs
2,894,2,male,62.0,0,0,240276,9.6875,NaN,Q,Mr
3,895,3,male,27.0,0,0,315154,8.6625,NaN,S,Mr
4,896,3,female,22.0,1,1,3101298,12.2875,NaN,S,Mrs


In [22]:
X_train = dataset[~dataset.Survived.isnull()].drop('Survived', axis=1)
y_train = dataset[~dataset.Survived.isnull()]['Survived']
X_test = dataset[dataset.Survived.isnull()].drop('Survived', axis=1)
# y_test = dataset[dataset.Survived.isnull()]['Survived'] ## should just be NaNs

In [23]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=42)

clf.fit(X_train, y_train)

C:\Users\henry\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=42, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
y_test = clf.predict(X_test)

In [34]:
test_survived = pd.Series(list(map(int, y_test)), name='Survived')
results = pd.concat([X_test['PassengerId'], test_survived], axis=1)
results.to_csv("results.csv", index=False)

In [35]:
results.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
